In [ ]:
import pandas as pd
import numpy as np
import logging
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import databricks.koalas as ks

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Step 1: Load Dataset (Open Source)
logging.info("Loading telecom customer churn dataset...")
kaggle_data_path = "telecom_churn_data.csv" 
try:
    df = pd.read_csv(kaggle_data_path)
    logging.info("Dataset loaded successfully.")
except FileNotFoundError:
    logging.error(f"File {kaggle_data_path} not found. Please check the file path.")
    sys.exit(1)

# Data Preprocessing
logging.info("Preprocessing data...")
required_columns = ['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    logging.error(f"Missing columns in dataset: {missing_columns}")
    sys.exit(1)

df = df[required_columns].dropna()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
logging.info("Data cleaned and transformed successfully.")

# Step 2: Store Data in Databricks (Koalas & SQL)
df_koalas = ks.from_pandas(df)
logging.info("Data converted to Koalas DataFrame for Databricks compatibility.")
engine = create_engine('sqlite:///telecom_churn.db')
df.to_sql('churn_data', con=engine, if_exists='replace', index=False)
logging.info("Data stored in SQLite database.")

# Step 3: SQL Queries for Customer Segmentation
query = "SELECT customerID, tenure, MonthlyCharges, Churn FROM churn_data WHERE Churn = 1"
df_sql = pd.read_sql(query, con=engine)
logging.info("Executed SQL query successfully.")
print(df_sql.head())

# Step 4: Train Machine Learning Model
X = df[['tenure', 'MonthlyCharges', 'TotalCharges']]
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logging.info("Training XGBoost model...")
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
logging.info(f'Model Accuracy: {accuracy * 100:.2f}%')

# Step 5: Model Evaluation
logging.info("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Step 6: Close Database Connection
engine.dispose()
logging.info("Database connection closed.")
